In [1]:
type(open('.gitignore', 'r'))

_io.TextIOWrapper

In [12]:
import influxdb_client, os
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from PyTracy.SECRETS import INFLUXDB_TOKEN

token = INFLUXDB_TOKEN
org = "freakingrocky"
url = "http://localhost:8086"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)


In [15]:
import time

bucket="test"

write_api = write_client.write_api(write_options=SYNCHRONOUS)
   
for value in range(5):
  point = (
    Point("measurement1")
    .tag("tagname1", "tagvalue1")
    .field("field1", value)
  )
  write_api.write(bucket=bucket, org="freakingrocky", record=point)
  time.sleep(1) # separate points by 1 second


In [37]:
import pandas as pd

query_api = write_client.query_api()

query = """from(bucket: "x")
 |> range(start: -10m)
 |> filter(fn: (r) => r._measurement == "measurement1")"""
tables = query_api.query(query, org="freakingrocky")

for table in tables:
  for record in table.records:
    print(record)


ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'Vary': 'Accept-Encoding', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.6', 'X-Platform-Error-Code': 'not found', 'Date': 'Thu, 09 May 2024 15:07:41 GMT', 'Transfer-Encoding': 'chunked'})
HTTP response body: b'{"code":"not found","message":"failed to initialize execute state: could not find bucket \\"x\\""}'


In [18]:
print(tables)

[<FluxTable: 9 columns, 5 records>]


In [21]:
pd.DataFrame(tables)

,0,1,2,3,4
0,"FluxRecord() table: 0, {'result': '_result', '...","FluxRecord() table: 0, {'result': '_result', '...","FluxRecord() table: 0, {'result': '_result', '...","FluxRecord() table: 0, {'result': '_result', '...","FluxRecord() table: 0, {'result': '_result', '..."


In [28]:
for table in tables:
    for record in table.records:
        print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 5, 9, 14, 43, 52, 658713, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 5, 9, 14, 53, 52, 658713, tzinfo=tzutc()), '_time': datetime.datetime(2024, 5, 9, 14, 53, 24, 451078, tzinfo=tzutc()), '_value': 0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 5, 9, 14, 43, 52, 658713, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 5, 9, 14, 53, 52, 658713, tzinfo=tzutc()), '_time': datetime.datetime(2024, 5, 9, 14, 53, 25, 489330, tzinfo=tzutc()), '_value': 1, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 5, 9, 14, 43, 52, 658713, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 5, 9, 14, 53, 52, 658713, tzinfo=tzutc()), '_time': datetime.datetime(2024, 5, 9,

In [29]:
import polars
data_frame = query_api.query_data_frame(query)
polars_data = polars.from_pandas(data_frame)


c:\Python311\Lib\site-packages\influxdb_client\client\warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    from(bucket: "test")
 |> range(start: -10m)
 |> filter(fn: (r) => r._measurement == "measurement1") |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


In [31]:
polars_data

result,table,_start,_stop,_time,_value,_field,_measurement,tagname1
str,i64,"datetime[ns, UTC]","datetime[ns, UTC]","datetime[ns, UTC]",i64,str,str,str
"""_result""",0,2024-05-09 14:46:13.987689 UTC,2024-05-09 14:56:13.987689 UTC,2024-05-09 14:53:24.451078 UTC,0,"""field1""","""measurement1""","""tagvalue1"""
"""_result""",0,2024-05-09 14:46:13.987689 UTC,2024-05-09 14:56:13.987689 UTC,2024-05-09 14:53:25.489330 UTC,1,"""field1""","""measurement1""","""tagvalue1"""
"""_result""",0,2024-05-09 14:46:13.987689 UTC,2024-05-09 14:56:13.987689 UTC,2024-05-09 14:53:26.526499 UTC,2,"""field1""","""measurement1""","""tagvalue1"""
"""_result""",0,2024-05-09 14:46:13.987689 UTC,2024-05-09 14:56:13.987689 UTC,2024-05-09 14:53:27.532639 UTC,3,"""field1""","""measurement1""","""tagvalue1"""
"""_result""",0,2024-05-09 14:46:13.987689 UTC,2024-05-09 14:56:13.987689 UTC,2024-05-09 14:53:28.548467 UTC,4,"""field1""","""measurement1""","""tagvalue1"""


In [36]:
query_api = write_client.query_api()

query = """from(bucket: "test")
  |> range(start: -10m)
  |> filter(fn: (r) => r._measurement == "measurement1")
  |> mean()"""
data_frame = query_api.query_data_frame(query)
polars_data = polars.from_pandas(data_frame)
polars_data

c:\Python311\Lib\site-packages\influxdb_client\client\warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    from(bucket: "test")
  |> range(start: -10m)
  |> filter(fn: (r) => r._measurement == "measurement1")
  |> mean() |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


result,table,_start,_stop,_value,_field,_measurement,tagname1
str,i64,"datetime[ns, UTC]","datetime[ns, UTC]",f64,str,str,str
"""_result""",0,2024-05-09 14:48:04.932175 UTC,2024-05-09 14:58:04.932175 UTC,2.0,"""field1""","""measurement1""","""tagvalue1"""


In [33]:
polars_data

shape: (0, 0)
┌┐
╞╡
└┘